The following steps need to be followed for setting up FlocktMTL with Jupyter

#### Step 1: Store API secrets in a .env file, and load it in jupyter
We store the OpenAI API key in our .env file for the tutorial. and use the python-dotenv package
to read this secret. Any other suitable method can also be used

#### Step 2: Install DuckDB for python. 
For FlockMTL v0.1.0 "Schwartz Deli", DuckDB version 1.1.1 is required

#### Step 3: Create DuckDB database.
We use a temporary in-memory database. Persistent database can also be used. Please
see https://duckdb.org/docs/connect/overview.html for detailed methods to create databases

#### Step 4: Install FlockMTL
FlockMTL needs to be installed once per new DuckDB installation. DuckDB installs the latest
release from DuckDB Community extensions (https://community-extensions.duckdb.org/)

#### Step 5: Load FlockMTL
FlockMTL needs to be loaded every time when the Jupyter kernel restarts

#### Step 6: Use FlockMTL
After step 5, FlockMTL is installed and ready to use. We create a new table called product_reviews,
and use LLM function calls to extract semantic details from it

In [ ]:
#Step 1 Store API secrets in a .env file, and load it in jupyter
!pip install python-dotenv
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()


In [ ]:
#Step 2: Install DuckDB for python
!pip install duckdb==1.1.1
import duckdb
print(duckdb.__version__)


In [ ]:
#Step 3: Create DuckDB database.
con = duckdb.connect(':memory:', config={'allow_unsigned_extensions' : 'true'})



In [ ]:
#Step 4: Install FlockMTL
con.execute(f"INSTALL flockmtl")


In [ ]:
#Step 5: Load FlockMTL
con.execute(f"LOAD flockmtl")


#### A simple sentiment analysis on product reviews is defined below. The following features are highlighted
* FlockMTL default supported models
* Use of LLMs to perform analysis tasks on SQL tables


In [ ]:
#Step 6: Use FlockMTL
csv_path = 'products.csv' #add your own path
con.execute(f"CREATE TABLE product_reviews AS SELECT * FROM read_csv_auto('{csv_path}')");
con.execute ('SELECT * from product_reviews;').fetchall()

In [ ]:
#View the default models supported out of the box by FlockMTL
con.execute("GET MODELS;").fetchall()


In [ ]:

# First we create a new sentiment analysis prompt
sentiment_analysis_prompt = """
Analyze the sentiment of the following product review. Consider both the review text and the star rating. Provide a brief sentiment label (positive, negative, or neutral) and a short explanation for your decision.

Review: {review}
Star Rating: {rating}

Output your response in the following JSON format:
{
    "sentiment": "positive/negative/neutral",
    "explanation": "Brief explanation of the sentiment analysis"
}
"""

# Use an f-string to insert the prompt directly into the query
sentiment_analysis_prompt_query = f"""
    CREATE PROMPT ('sentiment-analysis', '{sentiment_analysis_prompt}');
"""

con.execute (sentiment_analysis_prompt_query)



# Now we use the new prompt for analytical analysis
query = """
CREATE TABLE sentiment_analysis AS
WITH sentiment_analysis AS (
    SELECT 
        ProductID as product_id, 
        ID AS review_id,
        Review AS review_text, 
        Rating AS star_rating, 
        llm_complete_json('sentiment-analysis', 'default', {'review': review_text, 'rating': star_rating}) AS sentiment_json
    FROM 
        product_reviews
)
SELECT * 
FROM sentiment_analysis;
"""

# Run the query and fetch results
results = con.execute(query).fetchall()

In [ ]:
#Print newly created table
query = "select * from sentiment_analysis;"
results = con.execute(query).fetchall()

for row in results:
    print(row)

In [ ]:
# Define the is-high-impact-review prompt
is_high_impact_review_prompt = """
Determine if the given review is a high-impact review that provides valuable insights. Consider the following factors:

1. Sentiment: {sentiment}
2. Star Rating: {rating}
3. Review Length: {review_length}

A high-impact review typically has:

- A strong sentiment (very positive or very negative)
- An extreme rating (1-2 or 4-5 stars)
- Sufficient length to provide detailed feedback (usually more than 50 words)

Output your decision as a boolean true or false.
"""

# Create the SQL query to register the is-high-impact-review prompt
is_high_impact_review_prompt_query = f"""
    CREATE PROMPT ('is-high-impact-review', '{is_high_impact_review_prompt}');
"""

# Execute the query to create the prompt
con.execute(is_high_impact_review_prompt_query)


In [ ]:
query = """
CREATE TABLE filtered_reviews AS
WITH filtered_reviews AS (
    SELECT 
        * 
    FROM 
        sentiment_analysis 
    WHERE 
        llm_filter('is-high-impact-review', 'gpt-4o', {
            'sentiment': sentiment_json, 
            'rating': star_rating, 
            'review_length': LENGTH(review_text)
        })
)
SELECT * 
FROM filtered_reviews;
"""

# Run the query and fetch results
results = con.execute(query).fetchall()

for row in results:
    print(row)

In [ ]:
#Print newly created table
query = "select * from filtered_reviews;"
results = con.execute(query).fetchall()

for row in results:
    print(row)

In [ ]:
# Define the extract themes prompt
extract_themes_prompt = """
    Analyze the following sentiment analysis JSON. Identify and extract key themes or topics discussed in the product review. Output the themes in a JSON array format.
    
    Sentiment Analysis JSON: {{sentiment_json}}
    
    Output your response in the following JSON format:
    {{
        "themes": ["theme1", "theme2", "theme3"]
    }}
"""
       
# Create the SQL query to register the prompt
extract_themes_prompt_query = f"""
    CREATE PROMPT ('extract-themes' , '{extract_themes_prompt}');
"""

# Execute the query to create the prompt
results = con.execute(extract_themes_prompt_query)


In [ ]:
query = """
CREATE TABLE themes_extracted AS
WITH themes_extracted AS (
    SELECT 
        product_id, 
        review_id, 
        review_text,
        star_rating,
        llm_complete_json('extract-themes', 'gpt-4o', {'sentiment_json': sentiment_json}) AS themes
    FROM 
        filtered_reviews
)
SELECT * 
FROM themes_extracted;
"""
# Run the query and fetch results
results = con.execute(query).fetchall()

for row in results:
    print(row)

In [ ]:
#Print newly created table
query = "select * from themes_extracted;"
results = con.execute(query).fetchall()

for row in results:
    print(row)